In [ ]:
!unzip /content/DATA.zip -d .

In [2]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import Model
import cv2
from glob import glob
import random
import numpy as np


In [18]:
class DATALoader(tf.keras.utils.Sequence):
  def __init__(self):
    super()
    self.paths = []
    self.BatchSize = 20
    for path in glob("./Spectrogram/FAKE/*.png"):
      self.paths.append([path,1])
    for path in glob("./Spectrogram/REAL/*.png"):
      self.paths.append([path,0])
    random.shuffle(self.paths)
  def __len__(self):
    return len(self.paths)
  def image_load(self,path):
    return cv2.resize(cv2.imread(path),(256,256))/255
  def __getitem__(self,index):
    index = index%4
    x,y = [],[]
    for pathLabel in self.paths[index*self.BatchSize:(index+1)*self.BatchSize]:
      x.append(self.image_load(pathLabel[0]))
      y.append(pathLabel[1])
    return np.array(x),np.array(y)

In [19]:
dataLoader = DATALoader()

## MODEL

In [45]:
input_layer = layers.Input((256,256,3),name="input_layer")
x = layers.Conv2D(32,3,activation='relu',name="conv1")(input_layer)
x = layers.MaxPooling2D(name="max_pool1")(x)
x = layers.Conv2D(64,3,activation='relu',name="conv2")(x)
x = layers.MaxPooling2D(name="max_pool2")(x)
x = layers.Flatten()(x)
x = layers.Dense(30,activation='relu')(x)
x = layers.Dense(10,activation='relu')(x)
x = layers.Dense(1,activation='sigmoid')(x)

In [46]:
model = Model(input_layer,x)

In [47]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 254, 254, 32)      896       
                                                                 
 max_pool1 (MaxPooling2D)    (None, 127, 127, 32)      0         
                                                                 
 conv2 (Conv2D)              (None, 125, 125, 64)      18496     
                                                                 
 max_pool2 (MaxPooling2D)    (None, 62, 62, 64)        0         
                                                                 
 flatten_6 (Flatten)         (None, 246016)            0         
                                                                 
 dense_18 (Dense)            (None, 30)                7380

In [48]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [49]:
model.fit(dataLoader, epochs=2,batch_size=4,verbose=1)

Epoch 1/2
80/80 [==============================] - 26s 300ms/step - loss: 0.1868 - accuracy: 0.9419
Epoch 2/2
80/80 [==============================] - 27s 340ms/step - loss: 3.4664e-08 - accuracy: 1.0000


In [57]:
img = np.array([cv2.resize(cv2.imread("./Spectrogram/REAL/030f89ef-1685-4d4e-81b2-6f843b615ef7.png"),(256,256))])


In [58]:
model.predict(img)

1/1 [==============================] - 0s 19ms/step


array([[0.]], dtype=float32)

In [59]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
